# Telecom Growth Explanatory Data Analysis - TellCo

##  Import Libraries and Functions

In [22]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine
os.chdir('..')
from Db_connection.connection import PostgresConnection
from src.utils import missing_values_table, convert_bytes_to_megabytes

## Connect To PostgreSQL Database

In [23]:
db = PostgresConnection(dbname='telecom', user='postgres', password='postgres')
db.connect()

query = "SELECT * FROM xdr_data"
result = db.execute_query(query)


df = pd.DataFrame(result, columns=[desc[0] for desc in db.cursor.description])
print(df.head()) 
db.close_connection()

Connected to PostgreSQL database!
             Bearer Id            Start  Start ms              End  End ms  \
0 13114483460844900352   4/4/2019 12:01       770  4/25/2019 14:35     662   
1 13114483482878900224   4/9/2019 13:04       235   4/25/2019 8:15     606   
2 13114483484080500736   4/9/2019 17:42         1  4/25/2019 11:58     652   
3 13114483485442799616   4/10/2019 0:31       486   4/25/2019 7:36     171   
4 13114483499480700928  4/12/2019 20:10       565  4/25/2019 10:40     954   

   Dur. (ms)            IMSI  MSISDN/Number           IMEI  \
0    1823652 208201448079117    33664962239 35521209507511   
1    1365104 208201909211140    33681854413 35794009006359   
2    1361762 208200314458056    33760627129 35281510359387   
3    1321509 208201402342131    33750343200 35356610164913   
4    1089009 208201401415120    33699795932 35407009745539   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...            15854611  

# Task 1 - User Overview Analysis

## Understanding and cleaning the data

In [24]:
df.tail()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
149996,7277825670196679680,4/29/2019 7:28,451,4/30/2019 6:02,214,81230,208202201200072,33650688697,35483109451938,D20434A,...,16191667,11763428,17883703,19678161,526609673,9197207,3264510,13487416,57628851,574175259
149997,7349883264234609664,4/29/2019 7:28,483,4/30/2019 10:41,187,97970,208201908153249,33663449963,35660508296467,D10223C,...,13877234,8288284,19350146,21293148,626893062,4735033,712180387,2457758,39135081,666648844
149998,13114483573367300096,4/29/2019 7:28,283,4/30/2019 10:46,810,98249,208201711161187,33621890103,35721209870907,T51102A,...,22660510,1855903,9963942,5065760,553539484,13394316,121100856,11314729,34912224,592786405
149999,13114483573367300096,4/29/2019 7:28,696,4/30/2019 10:40,327,97910,208202101098075,33619622058,86186204011457,L88342B,...,8817106,8305402,3322253,13172589,352536971,2529475,814713113,1406930,29626096,371895920
150000,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,...,11634073,11009410,11626852,11001755,422044703,8288398,421100544,8264799,NaN,NaN


In [25]:
missing_values_df = missing_values_table(df)
print("Missing Values in df:")
print(missing_values_df)

Your selected dataframe has 55 columns.
There are 41 columns that have missing values.
Missing Values in df:
                                          Missing Values  % of Total Values  \
Nb of sec with 37500B < Vol UL                    130254                 87   
Nb of sec with 6250B < Vol UL < 37500B            111843                 75   
Nb of sec with 125000B < Vol DL                    97538                 65   
TCP UL Retrans. Vol (Bytes)                        96649                 64   
Nb of sec with 31250B < Vol DL < 125000B           93586                 62   
Nb of sec with 1250B < Vol UL < 6250B              92894                 62   
Nb of sec with 6250B < Vol DL < 31250B             88317                 59   
TCP DL Retrans. Vol (Bytes)                        88146                 59   
HTTP UL (Bytes)                                    81810                 54   
HTTP DL (Bytes)                                    81474                 54   
Avg RTT DL (ms)       

In [26]:
df.shape

(150001, 55)

In [27]:
df.drop_duplicates(inplace=True)

In [28]:
df.info

<bound method DataFrame.info of                   Bearer Id            Start  Start ms              End  \
0      13114483460844900352   4/4/2019 12:01       770  4/25/2019 14:35   
1      13114483482878900224   4/9/2019 13:04       235   4/25/2019 8:15   
2      13114483484080500736   4/9/2019 17:42         1  4/25/2019 11:58   
3      13114483485442799616   4/10/2019 0:31       486   4/25/2019 7:36   
4      13114483499480700928  4/12/2019 20:10       565  4/25/2019 10:40   
...                     ...              ...       ...              ...   
149996  7277825670196679680   4/29/2019 7:28       451   4/30/2019 6:02   
149997  7349883264234609664   4/29/2019 7:28       483  4/30/2019 10:41   
149998 13114483573367300096   4/29/2019 7:28       283  4/30/2019 10:46   
149999 13114483573367300096   4/29/2019 7:28       696  4/30/2019 10:40   
150000                  NaN             None       NaN             None   

        End ms  Dur. (ms)            IMSI  MSISDN/Number           

////////////////////////

In [29]:
pd.options.display.float_format = '{:.0f}'.format
from scripts.User_overview_analysis.analysis import *
from scripts.visualization.visualize_user_overview import *


## top 10 handsets used by the customers

In [31]:
handset_counts = df['Handset Type'].value_counts().sort_values(ascending = False)

handset_counts.head(10)

Handset Type
Huawei B528S-23A                19752
Apple iPhone 6S (A1688)          9419
Apple iPhone 6 (A1586)           9023
undefined                        8987
Apple iPhone 7 (A1778)           6326
Apple iPhone Se (A1723)          5187
Apple iPhone 8 (A1905)           4993
Apple iPhone Xr (A2105)          4568
Samsung Galaxy S8 (Sm-G950F)     4520
Apple iPhone X (A1901)           3813
Name: count, dtype: int64

## top 3 handset manufacturers

In [32]:
handset_counts = df['Handset Manufacturer'].value_counts().sort_values(ascending = False)

handset_counts.head(3)

Handset Manufacturer
Apple      59565
Samsung    40839
Huawei     34423
Name: count, dtype: int64

##  top 5 handsets per top 3 handset manufacturer

In [33]:
df[['Handset Manufacturer','Handset Type']].head(5)

,Handset Manufacturer,Handset Type
0,Samsung,Samsung Galaxy A5 Sm-A520F
1,Samsung,Samsung Galaxy J5 (Sm-J530)
2,Samsung,Samsung Galaxy A8 (2018)
3,undefined,undefined
4,Samsung,Samsung Sm-G390F


In [34]:
df_user_behavior = aggregate_user_behavior(df)
display(df_user_behavior)

,MSISDN/Number,Bearer_Id,Duration,Total_DL,Total_UL,SM_DL,SM_UL,Google_DL,Google_UL,Email_DL,Email_UL,YouTube_DL,YouTube_UL,Netflix_DL,Netflix_UL,Gaming_DL,Gaming_UL,Other_DL,Other_UL
0,33601001722,1,116720,842637466,36053108,2206504,25631,3337123,1051882,837400,493962,14900201,6724347,10265105,16915876,811091133,1367528,377096990,9473882
1,33601001754,1,181230,120755184,36104459,2598548,62017,4197697,1137166,2828821,478960,5324251,7107972,770569,10451194,105035298,14714780,279557701,2152370
2,33601002511,1,134969,556659663,39306820,3148004,47619,3343483,99643,2436500,768880,2137272,19196298,16525919,2827981,529068485,9759228,495086501,6607171
3,33601007832,1,49878,401993172,20327526,251469,28825,5937765,3740728,2178618,106052,4393123,2584198,1157362,784730,388074835,3051292,25248001,10031701
4,33601008617,2,18552,681565208,47140264,1430615,25656,6864334,2385474,1123904,528830,5169986,15596516,12485824,12115038,654490546,2908364,388821856,13580386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106851,33789996170,1,8810,687925212,26716429,234320,65863,6834178,697091,480946,525969,8294310,18353533,14754741,147797,657326717,3034642,291391283,3891534
106852,33789997247,1,140988,444575092,35732243,442214,56355,1472406,3957299,2513433,664,5596862,14254710,6929961,1601099,427620216,9383076,204636073,6479040
106853,3197020876596,1,877385,194828056,37295915,668596,46628,8572779,1865881,842279,678492,9839889,2120016,10340908,16251392,164563605,13485133,467678100,2848373
106854,337000037000919,1,253030,539634985,56652839,496337,25229,8215537,1615080,2989663,328919,16690728,20044212,9983678,20921364,501259042,8883653,144354057,4834382


In [35]:
missing_values = missing_values_table(df_user_behavior) 
display(missing_values)

Your selected dataframe has 19 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values,Dtype


In [36]:
metrics = analyze_basic_metrics(df_user_behavior)
display(metrics)

,Bearer_Id,Duration,Total_DL,Total_UL,SM_DL,SM_UL,Google_DL,Google_UL,Email_DL,Email_UL,YouTube_DL,YouTube_UL,Netflix_DL,Netflix_UL,Gaming_DL,Gaming_UL,Other_DL,Other_UL
count,106856,106856,106856,106856,106856,106856,106856,106856,106856,106856,106856,106856,106856,106856,106856,106856,106856,106856
mean,1,102695,455184477,41127097,1795947,32908,5750360,2055028,1792446,467059,11629307,11017120,11617520,10997794,422598897,8289507,421245976,8267681
std,1,72247,224790645,10386665,954210,17520,3043308,1095043,954200,248825,6182726,5849951,6194250,5854129,224620486,4408970,224157891,4392223
min,0,7142,8827082,2866892,12,0,207,3,97,14,53,106,42,35,2516,59,3290,156
25%,1,56410,276264920,34197060,1038284,19010,3346133,1185268,1036754,269604,6720295,6361144,6668676,6353316,243369920,4771741,243286828,4777439
50%,1,86400,456245462,41127015,1792984,32854,5740054,2054326,1792060,465892,11626465,11021903,11634538,10996365,423586168,8306379,421119218,8273797
75%,2,133912,633812078,48080206,2553164,46832,8181011,2923567,2551709,664105,16530491,15681806,16536794,15642394,601067887,11798529,599833498,11741376
max,18,1573420,902969616,78331311,3586007,65870,11462752,4121343,3586146,936418,23259098,22011962,23258336,22011955,843433310,16558794,843420874,16558816
